# Naive Bayes

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from qbstyles import mpl_style
import pandas as pd

## Background

### Bayes' Theorem
Bayes Theorem states the following relationship, 
$$\large P(A|B) = \frac{P(A)P(B|A)}{P(B)}$$

That is, given class variable $y$ and dependent feature vector $x_1$ through $x_n$:
$$\large P(y|x_1,...,x_n) = \frac{P(y)P(x_1,...,x_n|y)}{P(x_1,...,x_n)}$$

<br> Notation stolen from: <br>
https://scikit-learn.org/stable/modules/naive_bayes.html


### Naive
By definition of conditional probability (and ignoring the denominator for the moment), 
$$\large P(y)P(x_1,...,x_n | y) = P(x_1,...,x_n,y)$$
Using the chain rule to decompose, 
$$\large P(x_1,...x_n,y) = P(x_1|x_2,...,x_n,y)P(x_2|x_3,...,x_n,y)...P(x_{n-1}|x_n,y)P(x_n|y)P(y)$$
While this is correct, this is not entirely useful to us. A term such as $P(x_1|x_2,...,x_n,y) has so many conditions that it is not possible to calculate its probability accurately
<br><br>
Let us instead make the "naive" assumption that data is conditionally independant (of each other) given the class label. Naive Bayes is thus named naive for assumping independence where it might not actually exist. With this simplification, we can rewrite the chain rule probabilities using the conditional independence assumption. 
$$
\begin{align*}
P(x_1,...x_n,y) &= P(x_1|x_2,...,x_n,y)P(x_2|x_3,...,x_n,y)...P(x_{n-1}|x_n,y)P(x_n|y)P(y) \\
& \approx P(x_1|y)P(x_2|y)...P(x_{n-1}|y)P(x_n|y)P(y) \\
&= P(y) \prod_{i=1}^{n}P(x_i | y)
\end{align*}
$$
<br> Explanation derived from: <br>
https://courses.cs.washington.edu/courses/cse312/18sp/lectures/naive-bayes/naivebayesnotes.pdf



### Putting it together
With this simplification, our relationship can be simplified to
$$\large P(y|x_1,...,x_n) = \frac{P(y) \prod_{i=1}^{n}P(x_i | y)}{P(x_1,...,x_n)}$$

We can futher simplify this by dropping the denominator $P(x_1,...,x_n)$. This is possible because we will be computing $\frac{P(y) \prod_{i=1}^{n}P(x_i | y)}{P(x_1,...,x_n)}$ for each possible class, but $P(x_1,...,x_n)$ is constant regardless of class. Here, we are always asking about the most likely class for the same features, which must have the same probability $P(x_1,...,x_n)$. Therefore, 
$$\large P(y|x_1,...,x_n) \propto P(y) \prod_{i=1}^{n}P(x_i | y)$$

In other words, 
$$\large \hat{y} = \underset{y}{\mathrm{argmax}} \space P(y) \prod_{i=1}^{n}P(x_i | y)$$ 


* A quick notation note, $\space \large \hat{} \space$ means "our estimate of the correct class"
* $P(x_i | y)$ is the likelihood
* $P(y)$ is the priror probability of the class
* argmax is an operation that finds the argument that gives the maximum value from a target function. Argmax is most commonly used in machine learning for finding the class with the largest predicted probability

### Log space
To prevent floating point underflows (multiplying a set of small probabilities will likely result in floating point underflow, where the product will become too small to represent and be replaced with 0) and to increase speed, Naive Bayes calculations are commonly done in log space. Recall, $log(ab) = log(a) + log(b)$. Therefore, 
$$\large \hat{y} = \underset{y}{\mathrm{argmax}} \space log(P(y)) + \sum_{i=1}^{n}log(P(x_i|c))$$

### In Other Notation...
$$\large C_{NB} = C_{MAP} = \underset{c_j \in C}{\mathrm{argmax}} \space log(P(C_j)) +\sum_{i \in positions}log(P(x_i|c_j))$$
where C is class (Note, sometimes $C_{NB}$ is also $C_{MAP}$ where MAP is "maximum a posteriori", i.e the most likely class)

### Distribution of Data
The different naive Bayes classifiers differ mainly by the assumptions they make regarding the distribution of $P(x_i|y)$. We give some possibilities below:
- For real values, we can use the Gaussian distribution:<br>
$$
    \large p(\boldsymbol{x} | y = c, \boldsymbol{\theta}) = \prod_{i=1}^{D} \mathcal{N}(x_i| \mu_{ic}, \sigma_{ic}^2)
$$<br>
- For binary values, we can use a Bernouilli distribution, where $\mu_{ic}$ is the probability that feature $i$ occurs in class $c$:<br>
$$
    \large p(\boldsymbol{x} | y = c, \boldsymbol{\theta}) = \prod_{i=1}^{D} \text{Ber}(x_i | \mu_{ic}) 
$$<br>
- For categorical features, we can use a Multinouilli distribution, where $\boldsymbol{\mu}_{ic}$ is an histogram over the possible values for $x_i$ in class $c$:<br>
$$
    \large p(\boldsymbol{x} | y = c, \boldsymbol{\theta}) = \prod_{i=1}^{D}\text{Cat}(x_i | \boldsymbol{\mu}_{ic})
$$<br>

Source: http://blog.axelmendoza.fr/naive-bayes/alcohol/pytorch/eda/from-scratch/2020/09/17/Naive-Bayes-Classifier.html


## Algorithm

### Natural Language Processing
In a natural language processing task, our goal is to classify messages as either spam or not-spam (also known as ham). As such, our distribution of data is **multinomial**. (Data here is typically represented as word vector counts, thus making the distribution multinomial) NOTE, the algorithm implemented here **assumes a multinomial distribution of data**.

To estimate the probabilities, we first try the maximum likelihood estimate (MLE), which is simply the relative frequency and corresponds to the most likely value of each parameter given the trianing data. 

Let $N_y$ be the number of data in our training data with class $y$ and $N_{total}$ be the total number of data in our dataset. Then it follows that $$ \hat{P}(y) = \frac{N_y}{N_{total}}$$

To learn the probability $P(x_i|y)$ of feature $i$ appearing in a sample belonging to class $y$, we can make use of relative frequency counting. That is, $$\hat{P}(x_i|y) = \frac{N_{yi}}{N_y}$$ where $N_{yi} = \sum_{x \in T}x_i$ is the number of times feature $i$ appears in a sample of class $y$ in the training set $T$, and $N_y = \sum_{i=1}^{n}N_{yi}$ is the total count of all feature for class $y$


Notation Note: we write $\hat{P}$ for $P$ because we do not know the true values of the parameters $P(y)$ and $P(x_i|y)$, but rather we westimate them from the training set. 

Source: https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html (notation note)


### Smoothing
We however encouter a problem with MLE estimation. Consider a feature-class combination that did not occur in the training data. Take for example a feature $K$ that only occured in class $a$. The MLE estimates for other classes, say class $b$ would then be 0. $$\hat{P}(K|b) = 0$$ In such a case, our conditional probablitiy will be zero for class $b$ because we are multiplying the conditional probabiltiies for all terms (ref above first equation). This problem is due to spareseness, the training data is never large enough to represent the frequency of rare events adequately. 

The solution? Smoothing! This simply adds $\alpha$. We can rewrite our probability as follows: 
$$\hat{P}(x_i|y) = \frac{N_{yi} + \alpha}{N_y +\alpha n}$$ 
where $N_{yi} = \sum_{x \in T}x_i$ is the number of times feature $i$ appears in a sample of class $y$ in the training set $T$, and $N_y = \sum_{i=1}^{n}N_{yi}$ is the total count of all feature for class $y$, and $n$ is the number of features (in text classification, the size of the vocabulary) 

Note, $\alpha$ is a hyperparameter. Setting $\alpha=1$ is called Laplace smoothing, while $\alpha < 1$ is called Lidstone smoothing.

Source: https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html

### Training Algorithm
In conclusion, these are the training steps:
1) Group data according to the class label $c_i$
2) Compute the $log$ of the prior probability $\hat{P}(y_i)$ 
3) For each feature, compute the $log$ of $\hat{P}(x_i|y)$


In [2]:
'''
Recall, the naive bayes classifier is defined as: P(y|x) = P(x|y)P(y)
Note, P(y) and P(x|y) are calculated in the training phase
We return the log of it to prevent floating point underflow (reference above)
'''

def train(X,y, alpha=1.0):
    #number of samples
    sampleCount = X.shape[0]

    #group samples by class label
    separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]

    #calculate prior probability for each class, i.e P(y) = N_y/N_total
    classLogPrior = [np.log(len(i) / sampleCount) for i in separated] 

    #calculate P(x_i|y) with smoothing (we default to laplace smoothing, alpha = 1)
    #first calculate frequency
    count = np.array([np.array(i).sum(axis=0) for i in separated]) + alpha 
    #then calculate log probability 
    #[np.newaxis].T is simpy to transpore the array to allow for broadcasting.
    featureLogProb = np.log(count / count.sum(axis=1)[np.newaxis].T) 
    
    return classLogPrior, featureLogProb

### Prediction Algorithm
Recall, $$\large \hat{y} = \underset{y}{\mathrm{argmax}} \space log(P(y)) + \sum_{i=1}^{n}log(P(x_i|c))$$

To predict on new samples, 
- Add the log of each class, $log(P(y))$ with:
  - For each feature $k$:
    - Add the $log$ conditional probabilities calculated during training, $log(P(x_k|y_i))$ where $x_k$ is the value of the input on feature $k$
- Finally, return the highest probability $P(y_i|x)$ of all classes

Adapted from: https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html
<br> Also taken from: http://kenzotakahashi.github.io/naive-bayes-from-scratch-in-python.html

In [3]:
def predict(X, classLogPrior, featureLogProb):
    #calculate P(x|y)P(y)
    combinedLikelihood = [(featureLogProb * x).sum(axis=1) + classLogPrior for x in X]
    #return the class with the highest probability
    return np.argmax(combinedLikelihood, axis=1)

### Algorithmic Analysis
Training Time complexity of $O(nd+cd) = O(d)$ where:
* c the number of classes, in this case 1 since its binary classification
* n the number of instances/samples
* d the number of dimensions (atrributes)
* all it needs to do is computing the frequency of every feature value $d_i$ for each class.

Prediction Time complexity of $O(cd) = O(d)$ (remember, its a binary classifier, so $c = 1$)
* since you have to retrieve d feature values for each of the c classes. 

Space Complexity of $O(d)$
* Note, only decision trees are more compact

Source: https://www.inf.ed.ac.uk/teaching/courses/iaml/slides/naive-2x2.pdf

## Notes/Others
* Naive Bayes is very fast with low storage requirements
* Robust to Irrelevant Features
  * Irrevelevant features cancel each other without affecting results
* Very good in domains with many equally important features
* Optimal if the indepedence assumption hold

Real-world stuff:
* With enough data, classifier may not matter. (Brill and Banko on spelling correction graph)

Tweaking Performance:
* Domain-specific features and weights
* Collapse Terms
* Upweighting (Counting a word as if it occured twice)
  * Title Words (Cohen & Singer 1996)
  * First Sentence of each paragraph (Murata, 1999)
  * In sentences that contain title words (Ko et al, 2002)

Src: http://web.stanford.edu/~jurafsky/slp3/slides/7_NB.pdf


### Other Amazing Resources:
https://web.stanford.edu/~jurafsky/slp3/4.pdf <br>
https://cs229.stanford.edu/notes2021fall/cs229-notes2.pdf
